# Synthetic data generator, graphs and Gremlin in python
 Here we first create a list of assets; second we distribute randomly their ownership (fractional).

In [341]:
import random
import string

N = 10 # number of nodes
c = 0.2 # controll parameter for the connectivity U(0,c)


# code to generate a single random string
def get_random_string(length):    # Only upper case
    letters = string.ascii_uppercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    # print random string
    #print(result_str)
    return result_str

# code to generate a list of random strings. Return approx "N=length" elements because identical ones are dropped
def gen_random_asset_list(length,name_length):
    my_asset_list = []
    for i in range(length):
        asset_name = get_random_string(name_length)
        my_asset_list.append(asset_name)
    #print(my_asset_list)
    
    #remove identical entries by passing the generated list into a set and back to a list.
    new_asset_list = list(set(my_asset_list))
    return new_asset_list


In [342]:
# random edge probability per node

import numpy as np

assetlist = gen_random_asset_list(N,2)

# generate a list of probabilities to a edge to exists from each node:
node_edge_prob = []
count = 0
for i in assetlist:
    #node_edge_prob.append(np.random.uniform(0,c))
    #for the moment let us make it fixed
    node_edge_prob.append(c)
    print(i," ",node_edge_prob[count])
    count=count+1


    

OA   0.2
HT   0.2
EM   0.2
OQ   0.2
TI   0.2
WN   0.2
PV   0.2
QP   0.2
UA   0.2
DA   0.2


In [343]:
#generating the edges
edges = []

count = 0
for i in assetlist:
    for j in assetlist:
        if np.random.uniform(0,1) < node_edge_prob[count] and i!=j:
            edges.append((i,j))
    count=count+1

print(edges)

[('OA', 'TI'), ('OA', 'QP'), ('HT', 'TI'), ('HT', 'WN'), ('EM', 'OA'), ('EM', 'HT'), ('EM', 'QP'), ('OQ', 'HT'), ('OQ', 'PV'), ('OQ', 'UA'), ('OQ', 'DA'), ('TI', 'HT'), ('TI', 'OQ'), ('TI', 'WN'), ('TI', 'UA'), ('WN', 'HT'), ('PV', 'TI'), ('PV', 'DA'), ('UA', 'EM'), ('UA', 'PV'), ('UA', 'QP'), ('DA', 'OA'), ('DA', 'UA')]


In [344]:
#generate edges weigths: percentage over ownership

weights = [0]*len(edges)
#print(weights)
assets_value = []

#this will split the value in a given number of parts
def split_num_random(value,num_parts):
    dividing_points = []
    parts = []
    if num_parts == 1:
        parts.append(value)
        
    elif num_parts > 1:
        for i in range(0, num_parts-1):
            dividing_points.append(np.random.uniform(0,value))
        dividing_points.sort()
        #print(dividing_points)
        old_value = 0
        for i in range(len(dividing_points)):
            parts.append(dividing_points[i]-old_value)
            old_value = dividing_points[i]
        parts.append(value-old_value)
        #print(parts)
    return parts


#here edges and weights vectors must already be define
#this code associate divide the ownership into many parts, each corresponding to an in-vector to a node
def gen_edge_weights(node):
    # count number of connections for a node
    count = 0
    edges_index=[]
    
    for i in range(len(edges)):
        if edges[i][1] == node:
            edges_index.append(i)
            count=count+1
    
    #print(count)
    parts_local = split_num_random(1,count)
    #print(parts_local)
    
    count = 0
    for i in edges_index:
        #print(i)
        # weights are changed directly over the external variable
        weights[i]=parts_local[count]
        count=count+1

    return True

#weights and node values
for i in assetlist:
    #generate the weights of connections
    health=gen_edge_weights(i)
    #generate the node value (between 1 and 100)
    #assets_value.append(np.random.uniform(1,100))
    
    #for testing, all worth 100
    assets_value.append(100)
    
print(weights)
#print(assets_value)

[0.18381760391246205, 0.19613845237897198, 0.08999403978327158, 0.634760995747976, 0.5009336768398839, 0.14139739993476352, 0.5687157448215241, 0.06735279684149442, 0.4208198566930548, 0.1552061056383358, 0.4877740540600972, 0.18390373766441614, 1, 0.36523900425202405, 0.27063984646686845, 0.6073460655593259, 0.7261883563042664, 0.5122259459399028, 1, 0.5791801433069452, 0.23514580279950392, 0.4990663231601161, 0.5741540478947957]


In [345]:
# plot the graph
import networkx as nx
#import matplotlib.pyplot as plt
from pyvis.network import Network

G=nx.DiGraph()
#G.add_edges_from(edges)
for i in range(len(assetlist)):
    G.add_node(assetlist[i],value=assets_value[i])    
for i in range(len(edges)):
    G.add_edge(edges[i][0],edges[i][1], label=str(weights[i])[0:5])
   # print(edges[i][0]," ",edges[i][1]," ",str(weights[i]))

    
print(G)
net = Network(directed =True,notebook=True, cdn_resources='remote')
net.from_nx(G)
net.repulsion()
net.show("myGraph.html")


DiGraph with 10 nodes and 23 edges
myGraph.html


In [346]:
# setting an output to be loaded in Gremling as a dataset
# here I will format the output in the graphML standard
import os
import time

G2=nx.DiGraph()
for i in range(len(assetlist)):
    G2.add_node(assetlist[i],value=assets_value[i])
    
for i in range(len(edges)):
    G2.add_edge(edges[i][0],edges[i][1],fraction=weights[i])

# name it with .xml extention instead of .graphml, the g.io() loading function does not to seem to recognize it otherwise
time_stamp = time.strftime("%d-%H%M%S") 
file_name = "D:\\gitrepos\\assetnet\\datagen\data\\graph_"+time_stamp +".xml"
output_file = open(file_name,"w")
nx.write_graphml_lxml(G2,file_name)
output_file.close()

In [347]:
# setup the connection to the Gremlin server
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *

# python and Gremlin commands overlap in their naming. This library will provide the mapping to resolve that
from gremlin_python.process.traversal import Barrier, Bindings, Cardinality, Column, Direction, Operator, Order, P, Pop, Scope, T, WithOptions

# Create a GraphTraversalSource which is the basis for all Gremlin traversals
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

#directly from python (not a notebook like jupter) do this
#g = traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

#only for notebooks, to avoit "loop already running" error, do this:
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
connection = DriverRemoteConnection('ws://localhost:8182/gremlin','g', transport_factory=lambda:AiohttpTransport(call_from_event_loop=True))

g = traversal().withRemote(connection)
g.V().drop().iterate()

[['V'], ['drop'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [348]:
# using a test graph to compare results witht he gremlin console: later use the path to the latest graph
#g.io("D:\\gitrepos\\assetnet\\gremlinTest\\graph_test_renamed.xml").read().iterate()
g.io(file_name).read().iterate()

[['io', 'D:\\gitrepos\\assetnet\\datagen\\data\\graph_31-230338.xml'], ['read'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [350]:
lista= g.V("HT").out().id_().toList()
lista

['TI', 'WN']

In [351]:
lista_init_values= g.V("HT").outE().values("fraction").toList()
lista_init_values

[0.08999403978327158, 0.634760995747976]

In [352]:
mydic = {}
for i in range(len(lista)):
    mydic[lista[i]] = lista_init_values[i]
print(mydic)

{'TI': 0.08999403978327158, 'WN': 0.634760995747976}


In [355]:
from gremlin_python.process.traversal import Barrier, Bindings, Cardinality, Column, Direction, Operator, Order, P, Pop, Scope, T, WithOptions

lista3 = g.V(lista).until(__.has(T.id,"HT")).repeat(__.out().simplePath()).path().toList()
lista3

[path[v[TI], v[HT]],
 path[v[TI], v[OQ], v[HT]],
 path[v[TI], v[WN], v[HT]],
 path[v[TI], v[UA], v[EM], v[HT]],
 path[v[TI], v[OQ], v[UA], v[EM], v[HT]],
 path[v[TI], v[OQ], v[DA], v[UA], v[EM], v[HT]],
 path[v[TI], v[OQ], v[PV], v[DA], v[UA], v[EM], v[HT]],
 path[v[WN], v[HT]]]

In [356]:
# multiply the weights of connections trough the path
lista5 = g.withSack(1).V(lista)\
        .until(__.has(T.id,"HT")).repeat(__.outE().sack(Operator.mult).by("fraction").inV().simplePath())\
        .sack().limit(5).toList()
lista5

[0.18390373766441614,
 0.06735279684149442,
 0.22182647222127272,
 0.03826777060915879,
 0.021945739791260924]

In [357]:
lista_first_element = []
lista_first_value = []
for i in lista3:
    lista_first_element.append(i[0])
    val = g.V(i[0]).id_().toList()
    # val is a list with one string, not a string, so use val[0] to get a simple string
    lista_first_value.append(mydic[val[0]])
print(lista_first_value)
    

[0.08999403978327158, 0.08999403978327158, 0.08999403978327158, 0.08999403978327158, 0.08999403978327158, 0.08999403978327158, 0.08999403978327158, 0.634760995747976]


In [358]:
finalValues = []
for i in range(len(lista5)):
    finalValues.append(lista5[i]*lista_first_value[i])
print(finalValues)

sum(finalValues)

[0.01655024028366381, 0.006061350278468058, 0.019963060366064007, 0.0034438712706177477, 0.001974985779848062]


0.04799350797866168

In [40]:
# The connection should be closed on shut down to close open connections with connection.close()
connection.close()